In [25]:
import numpy as np
import pandas as pd
import gzip
from collections import defaultdict
from tqdm import tqdm
import itertools

In [60]:
ls_breed = ['Angus', 'Hereford', 'Charolais', 'Simmental', 'Limousin']

In [77]:
def extract_lglr_s_run(run, breed, directory):
    pos2lglr = {}
    pos2s = {}
    ls_lglr = []
    ls_s = []
    ls_pos = []
    st_miss = set()
    with open(f'{directory}/log/clues_run{run}_{breed}.e', 'r') as f_e:
        for line in f_e:
            if line.startswith('FileNotFoundError'):
                ls_line = line.strip().split('/')
                pos = ls_line[-2]
                st_miss.add(pos)
    
    with open(f'{directory}/log/clues_run{run}_{breed}.o', 'r') as f_o:
        for line in f_o:
            if line.startswith('split'):
                pos = line.strip().split(f'{breed}_')[-1]
                ls_pos.append(pos)
            elif line.strip().startswith('logLR'):
                lglr = float(line.strip().split()[-1])
                ls_lglr.append(lglr)
            elif line.strip().startswith('0-167'):
                s = float(line.strip().split()[-1])
                ls_s.append(s)
    for pos in st_miss:
        ls_pos.remove(pos)
    return ls_pos, ls_lglr, ls_s

In [91]:
def extract_lglr_s(directory, seed, ls_breeds):
    breed2site2lglr = {}
    breed2site2s = {}
    for breed in ls_breeds:
        breed2site2lglr[breed] = {}
        breed2site2s[breed] = {}
    for breed, run in tqdm(itertools.product(ls_breeds, range(1, 721)), total=5 * 720):
        ls_pos, ls_lglr, ls_s = extract_lglr_s_run(run, breed, directory)
        for pos, lglr, s in zip(ls_pos, ls_lglr, ls_s):
            # df_lgLR.loc[breed, pos] = lglr
            # df_s.loc[breed, pos] = s
            breed2site2lglr[breed][pos] = lglr
            breed2site2s[breed][pos] = s
    
    return breed2site2lglr, breed2site2s

In [79]:
def extract_lglr_s_toDataFrame(directory, seed, ls_breeds, ls_all_sites):
    breed2site2lglr, breed2site2s = extract_lglr_s(directory, seed, ls_breeds)
    breed2lglr = defaultdict(list)
    breed2s = defaultdict(list)
    ## Record lgLR and s values in the order of ls_all_sites, with missing values as None
    for breed in ls_breeds:
        for site in ls_all_sites:
            breed2lglr[breed].append(breed2site2lglr[breed].get(site, np.nan))
            breed2s[breed].append(breed2site2s[breed].get(site, np.nan))
    #to DataFrame
    df_lgLR = pd.DataFrame(
        np.array(list(breed2lglr.values())),
        index=ls_breeds, 
        columns=ls_all_sites
    )

    df_s = pd.DataFrame(
        np.array(list(breed2s.values())),
        index=ls_breeds, 
        columns=ls_all_sites
    )
    return df_lgLR, df_s

In [80]:
with gzip.open('chrAuto_fst_percentile99.95.tsv.gz', 'rt') as f_site:
    next(f_site)
    ls_sites = ['_'.join(line.strip().split()[:2]) for line in f_site]

In [92]:
for num, direct in zip([1, 10, 100], ['01.seed_1_sample_200_burnin_100', '02.seed_10_sample_200_burnin_100', '03.seed_100_sample_200_burnin_100']):
    df_lgLR, df_s = extract_lglr_s_toDataFrame(directory=direct, seed=num, ls_breeds=ls_breed, ls_all_sites=ls_sites)
    df_lgLR.to_csv(f'{direct}/fst_all_seed_{num}.logLR.tsv')
    df_s.to_csv(f'{direct}/fst_all_seed_{num}.s.tsv')

100%|██████████| 3600/3600 [03:52<00:00, 15.49it/s]
